# Jane Street January 2025 Puzzle

Link to current puzzle: https://www.janestreet.com/puzzles/current-puzzle/

Tentative archive link (currently not archived): https://www.janestreet.com/puzzles/current-puzzle/somewhat-square-sudoku-index/


In [7]:
from Sudoku import Sudoku

NULL_INT = 1

puzzle_st = (".......2."
             "........5"
             ".2......."
             "..0......"
             "........."
             "...2....."
             "....0...."
             ".....2..."
             "......5..")
puzzle_lst = []
for i in range(0, 81, 9):
    puzzle_lst.append([])
    for ch in puzzle_st[i:i+9]:
        if ch != '.':
            puzzle_lst[-1].append(int(ch))
        else:
            puzzle_lst[-1].append(NULL_INT)

puzzle_lst

[[1, 1, 1, 1, 1, 1, 1, 2, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 5],
 [1, 2, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 2, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 5, 1, 1]]

In [8]:
game = Sudoku(puzzle_lst, is_hyper_Sudoku=True) #null_digit = -1, valid_digits = set(range(10)))
print(game)

[1, 1, 1, 1, 1, 1, 1, 2, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 5]
[1, 2, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 2, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 2, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 5, 1, 1]



In [9]:
import test_Sudoku

runner = test_Sudoku.unittest.TextTestRunner(verbosity=2)
runner.run(test_Sudoku.suite())

check_unique_test (test_Sudoku.SudokuTest) ... ok
check_row_test (test_Sudoku.SudokuTest) ... ok
check_col_test (test_Sudoku.SudokuTest) ... ok
check_box_test (test_Sudoku.SudokuTest) ... ok
find_options_test (test_Sudoku.SudokuTest) ... ok
candidates_test (test_Sudoku.SudokuTest) ... ok
candidates_advanced (test_Sudoku.SudokuTest) ... ok
impossible_test (test_Sudoku.SudokuTest) ... ok
impossible_x_test (test_Sudoku.SudokuTest) ... ok
check_done (test_Sudoku.SudokuTest) ... ok
check_null_digit (test_Sudoku.SudokuTest) ... ok

----------------------------------------------------------------------
Ran 11 tests in 0.019s

OK


<unittest.runner.TextTestResult run=11 errors=0 failures=0>

In [10]:
import test_solver

runner = test_solver.unittest.TextTestRunner(verbosity=2)
runner.run(test_solver.suite())

check_solver (test_solver.SudokuSolverTest) ... ok
check_solved_puzzles (test_solver.SudokuSolverTest) ... ok
solve_X_sudoku (test_solver.SudokuSolverTest) ... ok
solve_hyper_sudoku (test_solver.SudokuSolverTest) ... ok
solve_hyper_sudoku_x (test_solver.SudokuSolverTest) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.230s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

In [54]:
from copy import deepcopy
from typing import List
import math
from Sudoku import Sudoku
from str2grid import str2grid, grid2str

def solve(game0: Sudoku, progress_factor=1.0, mx_gcd=None):
    global calls, depth_max, progress, progress_update, update_increment
    stacc = [(game0, 0)]
    while stacc:
        if verbose and progress%100 == 1:
            print("Solution Depth:",len(stacc), end=' ...\r')
            progress_update = (
                (progress//update_increment) + 1) * update_increment
            
        game, depth = stacc.pop()
        calls += 1
        depth_max = max(depth, depth_max)
        # solved = False
        # while not solved:
        solved = True  # assume solved
        edited = False  # if no edits, either done or stuck
        deadend = False
        cur_gcd = None
        pruned = False
        for i in range(game.n):
            
            # prune if we have at least two rows filled and their GCD is worse than the current GCD
            # all([s != game._null_digit for s in game.grid[i]]):
            if mx_gcd and all([s != game._null_digit for s in game.grid[i]]):
                row_asint = int(''.join([str(s) for s in game.grid[i]]))
                cur_gcd = row_asint if cur_gcd is None else math.gcd(cur_gcd, row_asint)
                if cur_gcd < mx_gcd:
                    pruned = True
                    # print("="*100 + '\n' + 'PRUNING\n' + '='*100)
                    break # prune branch before deadlock or solution found

            if deadend: break
            for j in range(game.n):
                if game.grid[i][j] == game._null_digit:
                    solved = False
                    options = game.candidates[i][j]
                    if len(options) == 0:
                        deadend = True #return False  # this call is going nowhere
                        break
                    elif len(options) == 1:  # Step 1
                        game.place_and_erase(
                            i, j, list(options)[0])  # Step 2
                        edited = True
                        # deadend = True
                        break
        
        if pruned:
            continue
        
        if edited:
            progress += 1
            stacc.append((game, depth))
            continue

        if solved:
            progress += progress_factor
            if not all_solutions:
                return grid2str(game.grid.copy())
            yield grid2str(game.grid.copy())
            row_ints = [int(''.join([str(s) for s in row])) for row in game.grid]
            mx_gcd = max(mx_gcd, math.gcd(*row_ints)) if mx_gcd is not None else math.gcd(*row_ints)

        # backtracking check point:
        nxt_guess, min_options = None, 10
        for i in range(9):
            for j in range(9):
                if len(game.candidates[i][j]) < min_options and len(game.candidates[i][j]) > 1:
                    nxt_guess, min_options = (i, j), len(game.candidates[i][j])
        if not nxt_guess:
            continue
        i, j = nxt_guess
        options = game.candidates[i][j]
        progress_factor *= (1/len(options))
        for y in options:
            game_next = deepcopy(game)
            game_next.place_and_erase(i, j, y)
            game_next.flush_candidates() # full grid cleaning
            stacc.append((game_next, depth+1))


calls, depth_max = 0, 0
progress, update_increment, progress_update = 0.0, 0.01, 0.01
solution_set = []
verbose = True
all_solutions=True

In [ ]:
# from solver import solve_sudoku
from typing import Iterable
from str2grid import str2grid, grid2str
import re

UNUSED = set(range(0, 10))
UNUSED -= {2, 5, 0, 1}
puzzle_st_template = (  ".......2."
                        "........5"
                        ".2......."
                        "..0......"
                        "........."
                        "...2....."
                        "....0...."
                        ".....2..."
                        "......5..")
def get_sudoku_solutions(outname='solutions.txt', exclude_set: Iterable = UNUSED):
    with open(outname, 'w') as f:
        mx_gcd = 27
        for n in exclude_set:
            exclude_iter = 0
            attempt = re.sub(r'\.', str(n), puzzle_st_template)
            grid = str2grid(attempt)
            cgame = Sudoku(grid, is_hyper_Sudoku=True, null_digit=n, valid_digits= set(range(10))-{n})
            print('Now finding solutions for\n', grid2str(grid))

            sols = solve(cgame, mx_gcd = mx_gcd)
            i = 1
            for s in sols:
                f.write(s + '\n')
                print(f"Found Grid Solution #{i:^9} excluding number {n}", end='\r')
                row_ints = [int(''.join([str(s) for s in row])) for row in str2grid(s)]
                mx_gcd = max(mx_gcd, math.gcd(*row_ints)) if mx_gcd is not None else math.gcd(*row_ints)
                print('\ncurrent max gcd:', mx_gcd)
                if i % 1_000 == 1:
                    print()
                    print('\n'.join(str(i) for i in (str2grid(s))))
                    print('\n' + '='*12)
                i += 1
            
                exclude_iter += 1
                if exclude_iter >= 20_000: # make sure we don't spend too long evaluating just one possible leavout number
                    break

get_sudoku_solutions('solutions5.txt', exclude_set=sorted(list(UNUSED), reverse=True))

Now finding solutions for
 999999929999999995929999999990999999999999999999299999999909999999992999999999599
Found Grid Solution #    1     excluding number 9
current max gcd: 27

[6, 5, 1, 7, 4, 0, 8, 2, 3]
[4, 8, 7, 6, 2, 3, 1, 0, 5]
[0, 2, 3, 5, 1, 8, 6, 4, 7]
[3, 1, 0, 4, 8, 7, 2, 5, 6]
[8, 4, 2, 0, 5, 6, 3, 7, 1]
[7, 6, 5, 2, 3, 1, 0, 8, 4]
[1, 7, 8, 3, 0, 5, 4, 6, 2]
[5, 0, 4, 1, 6, 2, 7, 3, 8]
[2, 3, 6, 8, 7, 4, 5, 1, 0]

Found Grid Solution #    2     excluding number 9
current max gcd: 27
Found Grid Solution #    3     excluding number 9
current max gcd: 27
Found Grid Solution #    4     excluding number 9
current max gcd: 27
Found Grid Solution #    5     excluding number 9
current max gcd: 27
Found Grid Solution #    6     excluding number 9
current max gcd: 27
Found Grid Solution #    7     excluding number 9
current max gcd: 27
Found Grid Solution #    8     excluding number 9
current max gcd: 27
Found Grid Solution #    9     excluding number 9
current max gcd: 27
Found G

# Find the solution with the greatest possible gcd

Read in each solution from 'solutions.txt', seperating by newlines and slicing into nine-by-nine numbers. Then find the gcd of each string, print the greatest string found thus far.

In [ ]:
import math
best_gcd = -1
best_gcd_board = ""

num = 1
with open('solutions4.txt', 'r') as f:
    sol = f.readline()
    while sol:
        if num % 100 == 0:
            print(f"Checking Grid Number: {num}", end='\r')
        num += 1
        rows = tuple([int(sol[i:i+9]) for i in range(0, 73, 9)])
        cur_gcd = math.gcd(*rows)
        if cur_gcd > best_gcd:
            best_gcd_board = rows
            best_gcd = cur_gcd
            print()
            print("\n".join([f"{str(n):>09s}" for n in rows]))
            print("Current Best GCD:", best_gcd)
        sol = f.readline()


361850724
487623105
025417683
830165247
652074831
714238056
578306412
106542378
243781560
Current Best GCD: 9
Checking Grid Number: 36800
651740823
487623105
023518647
310487256
842056371
765231084
178305462
504162738
236874510
Current Best GCD: 27
